# Import Statements

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import joblib
import optuna
import sklearn 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# Load Data

In [ ]:
# load data
train = pd.read_csv('../input/forest-cover-type-prediction/train.csv')
# view data
train.head()

In [ ]:
# remove ID column from set
train = train.iloc[:, 1:]
train.head()

# Add Features from Feature Engineering

In [ ]:
# add new features from feature engineering
train['Elev_to_Horizontal_Hyd'] = train.Elevation - 0.2 * train.Horizontal_Distance_To_Hydrology 
train['Elev_to_Horizontal_Road'] = train.Elevation - 0.05 * train.Horizontal_Distance_To_Roadways  
train['Elev_to_Verticle_Hyd'] = train.Elevation - train.Vertical_Distance_To_Hydrology 
train['Mean_Horizontal_Dist'] = (train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + 
                                 train.Horizontal_Distance_To_Roadways)/3 
train['Mean_Fire_Hydro'] = (train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology)/2

# Preprocessing

In [ ]:
# move target to first column
first_column = train.pop('Cover_Type')
  
# insert column using insert(position,column_name,first_column) function
train.insert(0, 'Cover_Type', first_column)
  
# view
train.head()

In [ ]:
# create cat, num, and y
X_cat = train.iloc[:, 11:55].values
B = train.iloc[:, 55:60]
A = train.iloc[:, 1:11]
X_num = pd.concat([A, B], axis = 1).values
y = train.iloc[:, 0].values

In [ ]:
# scale/standardizing numerical columns
# scaler object
scaler = StandardScaler()
# fit to training data
scaler.fit(X_num)
# scale num columns
X_num = scaler.transform(X_num)

# shape
print(f'Categorical Shape: {X_cat.shape}')
print(f'Numerical Shape: {X_num.shape}')
print(f'Label Shape: {y.shape}')

In [ ]:
# combine num and cat
X = np.hstack((X_num, X_cat))
print(X.shape)

# Various Models
The following sections include various models we built to predict the Forest Cover Type. We used GridSearchCV and Optuna to identify the optimal paramaters for each model. 

Our hyperparameter tuning can be found in the notebook: https://www.kaggle.com/emknowles/g2-forestcovertype-modelparams-notebook/

# Logistic Regression
Logistic Regression did not yield high accuracy, even after GridSearchCV and Optuna were used to find optimal parameters.

In [ ]:
# best model 

lr_model = LogisticRegression(random_state = 1, 
                              penalty = 'none', 
                              max_iter = 500, 
                              solver = 'saga')
lr_model.fit(X, y)
print(lr_model.score(X, y))

# Decision Tree
The best parameters from Optuna were {'max_depth': 19, 'min_samples_leaf': 1}

In [ ]:
# dt best model
dt_model = DecisionTreeClassifier(random_state = 1, 
                                  max_depth = 19, 
                                  min_samples_leaf = 1)
dt_model.fit(X, y)

# Random Forest
The best parameters from Optuna were {'n_estimators': 130, 'max_depth': 50, 'min_samples_leaf': 1}

In [ ]:
# best model
rf_model = RandomForestClassifier(random_state = 1, 
                                  n_estimators = 130, 
                                  max_depth = 50, 
                                  min_samples_leaf = 1)

rf_model.fit(X, y)

# Extra Tree Classifier
The best parameters from Optuna were {'max_depth': 33, 'min_samples_leaf': 1}

In [ ]:
# best model
tree_model = ExtraTreesClassifier(random_state = 1, 
                                  n_estimators = 1000, 
                                  max_depth = 33, 
                                  min_samples_leaf = 1)

tree_model.fit(X, y)

# Gradient Boosting Classifier
The best parameters from Optuna were {'max_depth': 19, 'min_samples_leaf': 16}

In [ ]:
%%time
# best model
gradb_model = GradientBoostingClassifier(random_state = 0,
                                         max_depth = 19, 
                                         min_samples_leaf = 16)

gradb_model.fit(X, y)

# Extreme Gradient Boosting Classifier
The best parameters from Optuna were {'max_depth': 14}

In [ ]:
%%time
# best model
xgb_model = XGBClassifier(random_state = 0,
                           max_depth = 14,
                           tree_method = 'gpu_hist')

xgb_model.fit(X, y)
xgb_model.score(X, y)

# Light Gradient Boosting Classifier
The best parameters from Optuna were {'num_leaves': 10, 'max_depth': 16}

In [ ]:
%%time
# best model
lgbm_model = LGBMClassifier(random_state = 0,
                            max_depth = 16,
                            num_leaves = 10)

lgbm_model.fit(X, y)

# Model Selection 

In [ ]:
%%time 
# create ensemble classifier 
ensemble_model = VotingClassifier(
    estimators = [('tree', tree_model), 
                  ('rf', rf_model), 
                  ('gradb', gradb_model), 
                  ('xgb', xgb_model)],
    voting = 'hard'
)

# fit
ensemble_model.fit(X, y)

# print training accuracy
print('Logistic Regression Accuracy', lr_model.score(X, y))
print('Decision Tree Accuracy', dt_model.score(X, y))
print('Random Forest Accuracy', rf_model.score(X, y))
print('Extra Trees Accuracy', tree_model.score(X, y))
print('Gradient Boosting Accuracy', gradb_model.score(X, y))
print('Extra Gradient Boosting Accuracy', xgb_model.score(X, y))
print('LightGBM Accuracy', lgbm_model.score(X, y))
print('Ensemble Accuracy:', ensemble_model.score(X, y))

# Save Preprocessor and Models

In [ ]:
# save scaler
joblib.dump(scaler, 'forest_cover_scaler_final.joblib')

In [ ]:
joblib.dump(rf_model, 'rf_model_final.joblib')
joblib.dump(tree_model, 'tree_model_final.joblib')
joblib.dump(gradb_model, 'gradb_model_final.joblib')
joblib.dump(xgb_model, 'xgb_model_final.joblib')
joblib.dump(lgbm_model, 'lgbm_model_final.joblib')
joblib.dump(ensemble_model, 'ensemble_model_final.joblib')
print('Model written to file.')